# Parse data from .txt or .csv files

In [76]:
def parse_txt(filename, oriented=True):
    """
    Parse data from txt file into dict python type.
    JSON serializable.
    """
    data = {}
    with open(filename) as file:
        
        line = file.readline()
        while line:
            
            # skip comments
            if line[0] == '#':
                line = file.readline()
                continue
            
            parent, child = line.split()
            parent = int(parent)
            child = int(child)
            
            # rows in data file can be duplicated
            if parent in data:
                if child not in data[parent]['linked']:
                    data[parent]['linked'].append(child)
                    data[parent]['degree'] += 1
            else:
                data[parent] = { 
                    'linked': [child],
                    'distances': {},
                    'degree': 1,
                    'centrality': 0,
                    'marked': False,
                    'active': True
                }
                
            if not oriented:
                if child in data:
                    if parent not in data[child]['linked']:
                        data[child]['linked'].append(parent)
                        data[child]['degree'] += 1

                else:    
                    data[child] = {
                        'linked': [parent],
                        'distances': {},
                        'degree': 1,
                        'centrality': 0,
                        'marked': False,
                        'active': True
                    }

            line = file.readline()

    return data

def parse(filename, oriented=True):
    if filename.split('.')[-1] == 'txt':
        return parse_txt(filename, oriented)

# Distance counting methods with modifications

In [97]:
from collections import deque 
import datetime as dt

def count_distance(vertice, data, h = -1, full=False, rollback=True, monitoring=False):
    """
    Counts distances form given vertice to all other in connectivity component that vertice belongs to.
    Also, of h parameter is provided, this method finds list of vertices which are h or less away from provided vertice.
    (As only distance from provided vertive becomes more then h method stops.)
    Based on BFS.
    vertice: index of source vertice
    data: dict with information about graph
    h: distance to closest vertices
    fill: complete BFS in spite of current distance > h
    """

    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    current_distance = 0
    centrality = 0
    vertices_number = 1
    nearest_vertices = []
    d0 = deque()
    d1 = deque()
    
    source_active = data[vertice]['active']
    
    d0.append(vertice)
    data[vertice]['marked'] = True
    
    operations_counter += 9
    
    while True:
        
        if (not d0 and not d1) or (h != -1 and current_distance > h and not full):
            operations_counter += 9
            break
        
        if current_distance % 2 == 0:
            
            v = d0.pop()
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d1.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            data[v]['distances'][vertice] = current_distance
            vertices_number += 1
            centrality += current_distance
            
            # set active to false if it vertice 'v' is too close to source vertice 'vertice'
            if h != -1 and current_distance <= h:
                data[v]['active'] = False
                nearest_vertices.append(v)
                
                operations_counter += 2
                
            # go to the next level of distance from the source vertice 'vertice'
            if not d0:
                current_distance += 1
                                
                operations_counter += 1
                
            operations_counter += 5 # number of "if"'s
                
        else:
            
            v = d1.pop()
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d0.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            data[v]['distances'][vertice] = current_distance
            vertices_number += 1
            centrality += current_distance
            
            # set active to false if it vertice 'v' is too close to source vertice 'vertice'
            if h != -1 and current_distance <= h:
                data[v]['active'] = False
                nearest_vertices.append(v)
                
                operations_counter += 3
            
            # go to the next level of distance from the source vertice 'vertice'
            if not d1:
                current_distance += 1
                
                operations_counter += 1
                
            operations_counter += 5 # number of "if"'s added
    
    # rollback data
    if rollback:
        for key, value in data.items():
            value['marked'] = False
    
    # set initial status
    data[vertice]['active'] = source_active
    
    operations_counter =+ 1
    
    if h == -1 or full:
        data[vertice]['centrality'] = centrality / vertices_number
        operations_counter += 4
        if monitoring:
            return operations_counter, dt.datetime.now() - timestamp_before_algorithm
    else:
        if monitoring:
            return nearest_vertices, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return nearest_vertices

    
def count_distances(data):
    """
    Counts distances between all nodes in graph.
    """
    for key, value in data.items():
        count_distance(key, data)
        

def bfs(source, stock, data, rollback=True, monitoring=False):
    """
    Count distance from source to stock without using landmarks.
    Pure BFS.
    """
    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    current_distance = 0
    d0 = deque()
    d1 = deque()
    
    d0.append(source)
    data[source]['marked'] = True
    
    operations_counter += 6
    
    while True:
        
        if not d0 and not d1:
            current_distance = -1
            
            operations_counter += 3
            break
        
        if current_distance % 2 == 0:
            
            v = d0.pop()
            
            if v == stock:
                break
                
            for i in data[v]['linked']: 
                if not data[i]['marked']:
                    d1.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            if not d0:
                current_distance += 1
                
                operations_counter += 1
                
            operations_counter += 3
                
        else:
            
            v = d1.pop()
            
            if v == stock:
                break
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d0.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            if not d1:
                current_distance += 1 
                
                operations_counter += 1
            
            operations_counter += 3
    
    if rollback:
        for key, value in data.items():
            value['marked'] = False

    if monitoring:
        return current_distance, operations_counter, dt.datetime.now() - timestamp_before_algorithm
    return current_distance

# Select landmarks using random, degree or centrality ranking

In [95]:
import random
from math import log
import datetime as dt

def select_landmarks(
    data: dict,
    number_of_landmarks = 0.1,
    ranking: str = 'degree',
    h: int = 1,
    rollback=False,
    monitoring=False
):
    """
    Select landmarks using constratined strategy with provided h and ranking parameters.
    Set h to -1 to get top 'number_of_landmarks' ranked vertices.
    Possible ranking: degree, random, closeness.
    Set rollback to True to roll back all vertices 'active' to True
    """
    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    data_items = data.items()
    graph_size = len(data_items)
    print('=' * 126)
    print('Graph size: ' + str(graph_size))
    
    number_of_landmarks = int(
        graph_size * (number_of_landmarks / 100)
    ) if number_of_landmarks >= 1 else int(
        graph_size * number_of_landmarks
    )
    
    print('=' * 126)
    print('Target number of landmarks: ' + str(number_of_landmarks))
    
    landmarks = []
    
    operations_counter += graph_size + 5
    
    if ranking == 'degree':
        
        data_sorted = sorted(data_items, key=lambda x: x[1]['degree'], reverse=True)
        
        operations_counter += graph_size * log(graph_size)
        
        if h == -1:
            operations_counter += number_of_landmarks + 1
            
            if monitoring:
                return [i[0] for i in data_sorted[:number_of_landmarks]], operations_counter, dt.datetime.now() - timestamp_before_algorithm
            
            return [i[0] for i in data_sorted[:number_of_landmarks]]
        
        while len(landmarks) < number_of_landmarks and data_sorted:
            v = data_sorted.pop(0)[0]
            
            # check if vertice 'v' is less than 'h' away from some previously selected landmark
            if not data[v]['active']:
                continue
                
            landmarks.append(v)
            _, operations, __ = count_distance(vertice=v, data=data, h=h, monitoring=True)
            
            operations_counter += operations + 3
    
    
    elif ranking == 'random':
        
        vertices = [i[0] for i in data_items]
        if h == -1:
            return [vertices[i] for i in random.sample(range(0,graph_size), number_of_landmarks)]

        operations_counter += len(data_items) + number_of_landmarks
        
        random.shuffle(vertices)
        
        operations_counter += len(vertices)
        
        while len(landmarks) < number_of_landmarks and vertices:
            
            v = vertices.pop(0)
            
            # check if vertice 'v' is less than 'h' away from some previously selected landmark
            if not data[v]['active']:
                continue
            
            landmarks.append(v)
            _, operations, __ = count_distance(vertice=v, data=data, h=h)
            
            operations_counter += operations + 3
            
    
    elif ranking == 'closeness':
        
        vertices = [i[0] for i in data_items]
        if h == -1:
            return [vertices[i] for i in random.sample(range(0,graph_size), number_of_landmarks)]

        operations_counter += len(data_items) + number_of_landmarks

        number_of_seeds = int(
            number_of_landmarks + (graph_size - number_of_landmarks) / 2
        ) if number_of_landmarks >= (graph_size / 2) else number_of_landmarks * 2
#         print('=' * 126)
#         print('Number of seeds: ' + str(number_of_seeds))
        
        random_indexes = random.sample(range(0,graph_size), number_of_seeds)
        seeds = [vertices[i] for i in random_indexes]
        
        operations_counter += number_of_seeds + len(random_indexes)
            
        for seed in seeds:
            _, operations, __ = count_distance(vertice=seed, data=data, h=h, full=True)
            operations_counter += operations

        seeds_sorted = [
            i[0] for i in sorted(
                [(j[0], j[1]['centrality']) for j in data.items() if j[0] in seeds],
                key= lambda x: x[1],
                reverse=True
            )
        ]
        
        landmarks.append(seeds_sorted.pop(0))
#         print(str(landmarks[0]) + ': ' + str(data[landmarks[0]]['active']))

        operations_counter += graph_size + len(seeds) * log(len(seeds))

        while len(landmarks) < number_of_landmarks:
            
            if not seeds_sorted:
#                 print('=' * 126)
#                 print('For unsortet vertices: ')
#                 print("Number of landmarks after iteration by sorted seeds: " + str(len(landmarks)))
#                 print('=' * 126)
                for key, value in data.items():
                    if len(landmarks) >= number_of_landmarks:
                        break
                    if value['active'] and key not in landmarks:
                        _, operations, __ = count_distance(vertice=key, data=data, h=h, full=True)
                        landmarks.append(key)
                        operations_counter += operations + 1
                    
                    operations_counter += 2
                break
            
            v = seeds_sorted.pop(0)
#             print(str(v) + ': ' + str(data[v]['active']))
            # check if vertice 'v' is less than 'h' away from some previously selected landmark
            if not data[v]['active']:
                continue
            
            landmarks.append(v) 
#         print('=' * 126)
       
            operations_counter += 3
        
            
    # data rollback
    if rollback:
        for key, value in data.items():
            value['active'] = True
                
    if monitoring:
        landmarks, operations_counter, dt.datetime.now() - timestamp_before_algorithm
    return landmarks

# Shortest path estimating via landmarks method

In [91]:
def shortest_path(source, stock, landmarks, data, estimation_strategy='geometrical_mean', monitoring=False):
    """
    Counts distance from source to stock using landmarks.
    For distance estimation geometric mean is used.
    There are 4 estimation strategies: geometrical mean, upper, lower and middle point. Geometrical mean is default.
    """
    
    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    source_distances = data[source]['distances']
    stock_distances = data[stock]['distances']
    
    L = -1
    U = 3 * graph_size
    
    operations_counter += 5
    
    for key, value in source_distances.items():
        
        temp = stock_distances.get(key, -1)
        # cheks if there are distance from stock to 'key' landmark
        if temp == -1:
            continue
        
        l = abs(value - temp)
        u = value + temp
        
        if l > L:
            L = l
            operations_counter += 1
        if u < U:
            U = u
            operations_counter += 1
            
        operations_counter += 9
            
    if L == -1 and U == 3 * graph_size:
        # this mean that source and stock are in different connectivity components
        
        operations_counter += 3
        
        if monitoring:
            return -1, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return -1
    
    # choose estimating type
    if estimation_strategy == 'geometrical_mean':
        result = (L * U) ** 0.5
    elif estimation_strategy == 'upper':
        result = U
    elif estimation_strategy == 'lower':
        result = L
    elif estimation_strategy == 'middle_point':
        result = (L + U) / 2
    else:
        result = (L * U) ** 0.5
    
    if monitoring:
        return result, operations_counter, dt.datetime.now() - timestamp_before_algorithm
    return result

In [96]:
data = parse('../datasets/CA-AstroPh.txt', oriented=False)
landmarks = select_landmarks(data, 0.05, 'degree')

print('=' * 126)
print('Number of calculated landmarks: ' + str(len(landmarks)))
print('=' * 126)
print('Landmarks: ')
print([{'vertice_id':key, 'active':value['active'], 'degree':value['degree'], 'centrality': value['centrality']} for key, value in data.items() if key in landmarks])
print('=' * 126)
for landmark in landmarks:
    assert data[landmark]['active'] != False

Graph size: 18772
Target number of landmarks: 938
Number of calculated landmarks: 938
Landmarks: 
[{'vertice_id': 276, 'active': True, 'degree': 31, 'centrality': 0}, {'vertice_id': 16442, 'active': True, 'degree': 30, 'centrality': 0}, {'vertice_id': 29829, 'active': True, 'degree': 13, 'centrality': 0}, {'vertice_id': 64124, 'active': True, 'degree': 27, 'centrality': 0}, {'vertice_id': 77915, 'active': True, 'degree': 8, 'centrality': 0}, {'vertice_id': 89308, 'active': True, 'degree': 32, 'centrality': 0}, {'vertice_id': 90506, 'active': True, 'degree': 49, 'centrality': 0}, {'vertice_id': 95070, 'active': True, 'degree': 9, 'centrality': 0}, {'vertice_id': 124023, 'active': True, 'degree': 30, 'centrality': 0}, {'vertice_id': 132445, 'active': True, 'degree': 132, 'centrality': 0}, {'vertice_id': 3365, 'active': True, 'degree': 103, 'centrality': 0}, {'vertice_id': 8445, 'active': True, 'degree': 27, 'centrality': 0}, {'vertice_id': 42512, 'active': True, 'degree': 9, 'centrality'

# Test results

In [ ]:
vertices_source = [i[0] for i in data_items]
vertices_stock = vertices_source.copy()
random.shuffle(vertices)
for source, stock in zip(vertices_source,vertices_stock):
    
    distance_bfs, operations_bfs, time_bfs = bfs(source, stock, data, monitoring=True)
    # TODO: check on differend landmarks and estimation algorithms
    distance_landmarks, operations_landmarks, time_landmarks = shortest_path(source, stock, data, monitoring=True)
    
    
    print('=' * 126)


In [ ]:
d = {
    1: {
        'linked': [2,3,4,10],
        'distances':{},
        'marked': False
    },
    2: {
        'linked': [1,5,6,7],
        'distances':{},
        'marked': False
    },
    3: {
        'linked': [1,8],
        'distances':{},
        'marked': False
    },
    4: {
        'linked': [1,9,10],
        'distances':{},
        'marked': False
    },
    5: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    6: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    7: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    8: {
        'linked': [3],
        'distances':{},
        'marked': False
    },
    9: {
        'linked': [4],
        'distances':{},
        'marked': False
    },
    10: {
        'linked': [4,1],
        'distances':{},
        'marked': False
    }
}

print(bfs(1, 3, d))